In [100]:
# code to work with Drupal API (more exploratory than anything else)
import pandas as pd
import numpy as np
import requests
import json

url = 'https://bondpapersproject.org//jsonapi/node/1250'
headers = {'Accept': 'application/vnd.api+json', 'Content-Type': 'application/vnd.api+json'}

In [101]:
grab = requests.get(url, headers=headers)
grab

<Response [403]>

I'm gonna leave all that jazz above, but I have a new and fun idea.

In [103]:
doc_catalog = pd.read_csv('document-catalog.csv')
uscities = pd.read_csv('uscities.csv')

In [104]:
doc_catalog.columns

Index(['ID', 'Title', 'Box', 'Document Length', 'Workflow Stage',
       'Image Filename', 'Image Identifier', 'Image URL', 'Document Body'],
      dtype='object')

In [105]:
doc_catalog.drop(['Box', 'Document Length', 'Workflow Stage',
                  'Image Filename', 'Image Identifier', 'Image URL'], axis=1, inplace=True)

In [106]:
doc_catalog

,ID,Title,Document Body
0,670,Undated Speech concerning Conditions of Black ...,[This speech includes pages with many differen...
1,667,Speeches making observations about the recent ...,"Now that the nation's voters — at least, 54% o..."
2,666,Speeches making observations about the recent ...,"Now that the nation's voters — at least, 54% o..."
3,665,Speeches making observations about the recent ...,"1\nNow that the nations voters — at least, 54%..."
4,663,Speech about the upcoming presidential electio...,The election approaching on November seventh i...
...,...,...,...
235,83,Speech concerning black bankers and businesses...,"Before I begin, let me tell you my financial q..."
236,84,Speech concerning challenges facing the Americ...,Now that the nation has changed had a change o...
237,85,Speech from the Alabama Voter Education Projec...,Thank you a great deal. As John Lewis has told...
238,86,Speech at Press Conference in Washington conce...,[Handwritten.]\none the point of today's conce...


In [107]:
uscities.drop(['population', 'zips', 'id'], axis=1, inplace=True)

In [108]:
uscities

,city,state_name,lat,lng
0,New York,New York,40.6943,-73.9249
1,Los Angeles,California,34.1141,-118.4068
2,Chicago,Illinois,41.8375,-87.6866
3,Miami,Florida,25.7840,-80.2101
4,Houston,Texas,29.7860,-95.3885
...,...,...,...,...
31115,Kohatk,Arizona,32.5781,-112.0032
31116,Ironville,Pennsylvania,40.6586,-78.2155
31117,Newkirk,New Mexico,35.0635,-104.2715
31118,Falcon Village,Texas,26.5652,-99.1341


In [109]:
uscities['City'] = uscities['city'] + ', ' + uscities['state_name']

In [110]:
cities = list(uscities.City)
city_pattern = '|'.join([f'\\b{city}\\b' for city in cities])

# Extract the city name using str.extract()
doc_catalog['City'] = doc_catalog['Title'].str.extract(f'({city_pattern})', expand=False)

print(doc_catalog)

      ID                                              Title  \
0    670  Undated Speech concerning Conditions of Black ...   
1    667  Speeches making observations about the recent ...   
2    666  Speeches making observations about the recent ...   
3    665  Speeches making observations about the recent ...   
4    663  Speech about the upcoming presidential electio...   
..   ...                                                ...   
235   83  Speech concerning black bankers and businesses...   
236   84  Speech concerning challenges facing the Americ...   
237   85  Speech from the Alabama Voter Education Projec...   
238   86  Speech at Press Conference in Washington conce...   
239   88  Speech on the health of the United States educ...   

                                         Document Body                    City  
0    [This speech includes pages with many differen...                     NaN  
1    Now that the nation's voters — at least, 54% o...                     NaN  


In [111]:
doc_catalog.City.unique()

array([nan, 'New Orleans, Louisiana', 'Atlanta, Georgia',
       'Sanford, Florida', 'Houston, Texas', 'Atlantic City, New Jersey',
       'Baltimore, Maryland', 'Montgomery, Alabama', 'Arlington, Texas',
       'Seattle, Washington', 'Los Angeles, California', 'Macon, Georgia',
       'Charlotte, North Carolina', 'Pittsburgh, Pennsylvania',
       'Ruleville, Mississippi', 'Greensboro, North Carolina',
       'Birmingham, Alabama', 'Cleveland, Ohio', 'Jackson, Mississippi',
       'St. Louis, Missouri', 'Chicago, Illinois',
       'Kansas City, Missouri'], dtype=object)

In [112]:
sum(doc_catalog.City.value_counts())

62

In [113]:
# doc_catalog['city_from_body'] = doc_catalog['Document Body'].str.extract(f'({city_pattern})', expand=False)

# print(doc_catalog)

In [114]:
# sum(doc_catalog.city_from_body.value_counts())

In [129]:
doc_catalog['ID'] = doc_catalog['ID'].astype('str')
doc_catalog['url'] = "http://bondpapersproject.org/node/" + doc_catalog['ID']
doc_catalog

,ID,Title,Document Body,City,url
0,670,Undated Speech concerning Conditions of Black ...,[This speech includes pages with many differen...,NaN,http://bondpapersproject.org/node/670
1,667,Speeches making observations about the recent ...,"Now that the nation's voters — at least, 54% o...",NaN,http://bondpapersproject.org/node/667
2,666,Speeches making observations about the recent ...,"Now that the nation's voters — at least, 54% o...",NaN,http://bondpapersproject.org/node/666
3,665,Speeches making observations about the recent ...,"1\nNow that the nations voters — at least, 54%...",NaN,http://bondpapersproject.org/node/665
4,663,Speech about the upcoming presidential electio...,The election approaching on November seventh i...,"New Orleans, Louisiana",http://bondpapersproject.org/node/663
...,...,...,...,...,...
235,83,Speech concerning black bankers and businesses...,"Before I begin, let me tell you my financial q...",NaN,http://bondpapersproject.org/node/83
236,84,Speech concerning challenges facing the Americ...,Now that the nation has changed had a change o...,NaN,http://bondpapersproject.org/node/84
237,85,Speech from the Alabama Voter Education Projec...,Thank you a great deal. As John Lewis has told...,NaN,http://bondpapersproject.org/node/85
238,86,Speech at Press Conference in Washington conce...,[Handwritten.]\none the point of today's conce...,NaN,http://bondpapersproject.org/node/86


In [131]:
doc_catalog['url'][1]

'http://bondpapersproject.org/node/667'

So I think the conclusion here is that location tagging ought to be done during proofreading. I wish there was an easy way to make that location tagging happen, especially vis a vis creating the locations, but maybe this can be a jumping off point.

Essentially, there is no foolproof way to do this, but if the location is in the title, we can create entries for those and then build out from there. And hopefully, we can then go in and scrape that location data and apply it to the map tile layer.

city_from_body seems fairly useless

In [135]:
grouped = doc_catalog.groupby('City').agg({'Title': lambda x: ' | '.join(x),
                                           'ID': 'count',
                                           'url': lambda x: ' | '.join(x)
                                           }).reset_index()
grouped

,City,Title,ID,url
0,"Arlington, Texas","Speech--""Strategies for Change"" The Texas Regi...",1,http://bondpapersproject.org/node/336
1,"Atlanta, Georgia",Speech about the roles of black students in ad...,25,http://bondpapersproject.org/node/618 | http:/...
2,"Atlantic City, New Jersey",Speech given at the Philadelphia Bar Associati...,1,http://bondpapersproject.org/node/470
3,"Baltimore, Maryland",Commencement Address at Morgan State Universit...,1,http://bondpapersproject.org/node/442
4,"Birmingham, Alabama",Speech concerning opposition to the War in Sou...,2,http://bondpapersproject.org/node/213 | http:/...
5,"Charlotte, North Carolina",Speech prepared for Black Democrats Conference...,2,http://bondpapersproject.org/node/311 | http:/...
6,"Chicago, Illinois",Speech concerning the 1972 Presidential Electi...,1,http://bondpapersproject.org/node/118
7,"Cleveland, Ohio",Remarks prepared for Morehouse College Alumni ...,3,http://bondpapersproject.org/node/157 | http:/...
8,"Greensboro, North Carolina",Remarks to the Triad Sickle Cell Anemia Founda...,1,http://bondpapersproject.org/node/237
9,"Houston, Texas","Speech delivered at the NAACP Convention, Geor...",4,http://bondpapersproject.org/node/521 | http:/...


In [143]:
merged = pd.merge(grouped, uscities, on='City', how='left')
merged

,City,Title,ID,url,city,state_name,lat,lng
0,"Arlington, Texas","Speech--""Strategies for Change"" The Texas Regi...",1,http://bondpapersproject.org/node/336,Arlington,Texas,32.6998,-97.1250
1,"Atlanta, Georgia",Speech about the roles of black students in ad...,25,http://bondpapersproject.org/node/618 | http:/...,Atlanta,Georgia,33.7628,-84.4220
2,"Atlantic City, New Jersey",Speech given at the Philadelphia Bar Associati...,1,http://bondpapersproject.org/node/470,Atlantic City,New Jersey,39.3797,-74.4527
3,"Baltimore, Maryland",Commencement Address at Morgan State Universit...,1,http://bondpapersproject.org/node/442,Baltimore,Maryland,39.3051,-76.6144
4,"Birmingham, Alabama",Speech concerning opposition to the War in Sou...,2,http://bondpapersproject.org/node/213 | http:/...,Birmingham,Alabama,33.5279,-86.7971
5,"Charlotte, North Carolina",Speech prepared for Black Democrats Conference...,2,http://bondpapersproject.org/node/311 | http:/...,Charlotte,North Carolina,35.2083,-80.8303
6,"Chicago, Illinois",Speech concerning the 1972 Presidential Electi...,1,http://bondpapersproject.org/node/118,Chicago,Illinois,41.8375,-87.6866
7,"Cleveland, Ohio",Remarks prepared for Morehouse College Alumni ...,3,http://bondpapersproject.org/node/157 | http:/...,Cleveland,Ohio,41.4764,-81.6805
8,"Greensboro, North Carolina",Remarks to the Triad Sickle Cell Anemia Founda...,1,http://bondpapersproject.org/node/237,Greensboro,North Carolina,36.0956,-79.8271
9,"Houston, Texas","Speech delivered at the NAACP Convention, Geor...",4,http://bondpapersproject.org/node/521 | http:/...,Houston,Texas,29.7860,-95.3885


In [145]:
drop_map = ['city', 'state_name']
merged.drop(drop_map, axis=1, inplace=True)
merged.rename({'ID':'count', 'Title':'Documents'}, axis=1, inplace=True)
merged

,City,Documents,count,url,lat,lng
0,"Arlington, Texas","Speech--""Strategies for Change"" The Texas Regi...",1,http://bondpapersproject.org/node/336,32.6998,-97.1250
1,"Atlanta, Georgia",Speech about the roles of black students in ad...,25,http://bondpapersproject.org/node/618 | http:/...,33.7628,-84.4220
2,"Atlantic City, New Jersey",Speech given at the Philadelphia Bar Associati...,1,http://bondpapersproject.org/node/470,39.3797,-74.4527
3,"Baltimore, Maryland",Commencement Address at Morgan State Universit...,1,http://bondpapersproject.org/node/442,39.3051,-76.6144
4,"Birmingham, Alabama",Speech concerning opposition to the War in Sou...,2,http://bondpapersproject.org/node/213 | http:/...,33.5279,-86.7971
5,"Charlotte, North Carolina",Speech prepared for Black Democrats Conference...,2,http://bondpapersproject.org/node/311 | http:/...,35.2083,-80.8303
6,"Chicago, Illinois",Speech concerning the 1972 Presidential Electi...,1,http://bondpapersproject.org/node/118,41.8375,-87.6866
7,"Cleveland, Ohio",Remarks prepared for Morehouse College Alumni ...,3,http://bondpapersproject.org/node/157 | http:/...,41.4764,-81.6805
8,"Greensboro, North Carolina",Remarks to the Triad Sickle Cell Anemia Founda...,1,http://bondpapersproject.org/node/237,36.0956,-79.8271
9,"Houston, Texas","Speech delivered at the NAACP Convention, Geor...",4,http://bondpapersproject.org/node/521 | http:/...,29.7860,-95.3885


Of course, the next question is how to hyperlink them all, but perhaps that is one for later. Not bad work today.

I have some thoughts on how to hyperlink, too. Definitely not nothing.

Ok, so, cannot create a content field with a list of hyperlinks and have it work. Individual hyperlinks do work though - could create links to each place name and have links to content

In [147]:
merged.to_csv('test_layer_2.csv')

alternatively:

In [181]:
merged_pt_2 = pd.merge(doc_catalog, uscities, on='City', how='left')
merged_pt_2.dropna(inplace=True)
merged_pt_2

,ID,Title,Document Body,City,url,city,state_name,lat,lng
4,663,Speech about the upcoming presidential electio...,The election approaching on November seventh i...,"New Orleans, Louisiana",http://bondpapersproject.org/node/663,New Orleans,Louisiana,30.0687,-89.9288
5,662,(Draft) Speech about the upcoming presidential...,Page 1\nJULIAN BOND\nThe election approaching ...,"New Orleans, Louisiana",http://bondpapersproject.org/node/662,New Orleans,Louisiana,30.0687,-89.9288
6,661,(Draft) Speech about the upcoming presidential...,[Handwritten on legal pad paper.]\n[The follow...,"New Orleans, Louisiana",http://bondpapersproject.org/node/661,New Orleans,Louisiana,30.0687,-89.9288
11,618,Speech about the roles of black students in ad...,1\n[Handwritten.]\nI have been asked to say a ...,"Atlanta, Georgia",http://bondpapersproject.org/node/618,Atlanta,Georgia,33.7628,-84.4220
16,613,"Speech concerning political action, black peop...",It is a great pleasure to be in Florida this e...,"Sanford, Florida",http://bondpapersproject.org/node/613,Sanford,Florida,28.7891,-81.2757
...,...,...,...,...,...,...,...,...,...
221,90,"Speech at the National Urban League Meeting, K...",The quadrennial occasion for American reaction...,"Kansas City, Missouri",http://bondpapersproject.org/node/90,Kansas City,Missouri,39.1238,-94.5541
226,95,Testimony before the Commission on Delegate Se...,I am pleased to testify on behalf of the refor...,"Atlanta, Georgia",http://bondpapersproject.org/node/95,Atlanta,Georgia,33.7628,-84.4220
227,96,Testimony before the Commission on Delegate Se...,"""Quotas"" is the code word for 1972. Like most ...","Atlanta, Georgia",http://bondpapersproject.org/node/96,Atlanta,Georgia,33.7628,-84.4220
228,97,Testimony before the Commission on Delegate Se...,"""Quotas"" is the code word for 1972. Like most ...","Atlanta, Georgia",http://bondpapersproject.org/node/97,Atlanta,Georgia,33.7628,-84.4220


In [185]:
drop_map_2 = drop_map + ['Document Body']
merged_pt_2.drop(drop_map_2, axis=1, inplace=True)
merged_pt_2

,ID,Title,City,url,lat,lng
4,663,Speech about the upcoming presidential electio...,"New Orleans, Louisiana",http://bondpapersproject.org/node/663,30.0687,-89.9288
5,662,(Draft) Speech about the upcoming presidential...,"New Orleans, Louisiana",http://bondpapersproject.org/node/662,30.0687,-89.9288
6,661,(Draft) Speech about the upcoming presidential...,"New Orleans, Louisiana",http://bondpapersproject.org/node/661,30.0687,-89.9288
11,618,Speech about the roles of black students in ad...,"Atlanta, Georgia",http://bondpapersproject.org/node/618,33.7628,-84.4220
16,613,"Speech concerning political action, black peop...","Sanford, Florida",http://bondpapersproject.org/node/613,28.7891,-81.2757
...,...,...,...,...,...,...
221,90,"Speech at the National Urban League Meeting, K...","Kansas City, Missouri",http://bondpapersproject.org/node/90,39.1238,-94.5541
226,95,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",http://bondpapersproject.org/node/95,33.7628,-84.4220
227,96,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",http://bondpapersproject.org/node/96,33.7628,-84.4220
228,97,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",http://bondpapersproject.org/node/97,33.7628,-84.4220


In [187]:
merged_pt_2.to_csv('test_layer_3.csv')

# Using all the speeches

In [189]:
catalog_extended = pd.read_csv('document-catalog_extended.csv')
catalog_extended

,ID,Title,Box,Document Length,Workflow Stage,Image Filename,Image Identifier,Image URL,Document Body
0,4375,Speech concerning Black Voters and Officials d...,Box 15 Folder 1,6.0,Cataloged,NaN,NaN,NaN,NaN
1,3261,Enclosure: Julian Bond's revisions to text abo...,Box 14 Folder 6,1.0,Cataloged,NaN,NaN,NaN,NaN
2,671,"Speech File Material: Speech Fragment, 1971?",Box 12 Folder 3,4.0,Transcribed,"PJB671_mss13347-b12-f3_0088.tif\n, PJB671_mss1...","3243642, 3243643, 3243644, 3243645",NaN,"(9)\nNearly 100 years ago, a group of black pe..."
3,670,Undated Speech concerning Conditions of Black ...,Box 12 Folder 3,17.0,Early Access,"PJB670_mss13347-b12-f3_0071.tif\n, PJB670_mss1...","3243646, 3243647, 3243648, 3243649, 3243650, 3...",NaN,[This speech includes pages with many differen...
4,668,Speech File Material: Undated Speech about the...,Box 12 Folder 3,10.0,Cataloged,"PJB668_mss13347-b12-f3_0058.tif\n, PJB668_mss1...","3243666, 3243667, 3243668, 3243669, 3243670, 3...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
433,88,Speech on the health of the United States educ...,Box 4 Folder 8,29.0,Early Access,"PJB88_0001.tif\n, PJB88_0002.tif\n, PJB88_0003...","2857526, 2857527, 2857528, 2857529, 2857530, 2...",NaN,Insert x [Handwritten pages contain the conten...
434,4,"""Collision Course in a Divided America""",NaN,NaN,Digitized,NaN,NaN,NaN,"Unnamed speaker:\nIn the past few years, espec..."
435,7,"Address at the ""New Directions in Civil Rights...",NaN,NaN,Transcribed,NaN,NaN,NaN,This is the twentieth anniversary year of the ...
436,11,"DUPLICATE DO NOT PUBLISH: ""Civil Rights and Wr...",NaN,NaN,Digitized,"1578302.tiff\n, 1578303.tiff","1578302, 1578303",NaN,The women’s movement was given new life in the...


In [191]:
catalog_extended.drop(['Box', 'Document Length',
                  'Image Filename', 'Image Identifier', 'Image URL'], axis=1, inplace=True)

In [193]:
catalog_extended['City'] = catalog_extended['Title'].str.extract(f'({city_pattern})', expand=False)

sum(catalog_extended.City.value_counts())

97

EDA unfortunately shows that it will be awfully difficult to extract the dates because they are not in the same pattern.

In [227]:
years = np.arange(1960, 2016)
years = years.astype('str')
years_list = list(years)
years_pattern = '|'.join([f'\\b{year}\\b' for year in years])
years_pattern

'\\b1960\\b|\\b1961\\b|\\b1962\\b|\\b1963\\b|\\b1964\\b|\\b1965\\b|\\b1966\\b|\\b1967\\b|\\b1968\\b|\\b1969\\b|\\b1970\\b|\\b1971\\b|\\b1972\\b|\\b1973\\b|\\b1974\\b|\\b1975\\b|\\b1976\\b|\\b1977\\b|\\b1978\\b|\\b1979\\b|\\b1980\\b|\\b1981\\b|\\b1982\\b|\\b1983\\b|\\b1984\\b|\\b1985\\b|\\b1986\\b|\\b1987\\b|\\b1988\\b|\\b1989\\b|\\b1990\\b|\\b1991\\b|\\b1992\\b|\\b1993\\b|\\b1994\\b|\\b1995\\b|\\b1996\\b|\\b1997\\b|\\b1998\\b|\\b1999\\b|\\b2000\\b|\\b2001\\b|\\b2002\\b|\\b2003\\b|\\b2004\\b|\\b2005\\b|\\b2006\\b|\\b2007\\b|\\b2008\\b|\\b2009\\b|\\b2010\\b|\\b2011\\b|\\b2012\\b|\\b2013\\b|\\b2014\\b|\\b2015\\b'

In [291]:
catalog_extended['Year'] = catalog_extended['Title'].str.extract(f'({years_pattern})', expand=False)
catalog_extended['Year'].value_counts()

Year
1977    17
1972     9
1975     8
1976     8
1971     6
1979     5
1970     4
1980     4
1981     3
1978     3
1973     3
1998     3
1991     2
1974     2
1983     2
2001     2
1989     2
1965     1
2003     1
2002     1
2000     1
1996     1
1999     1
1994     1
1988     1
1968     1
1995     1
1987     1
Name: count, dtype: int64

In [293]:
catalog_extended.dropna(inplace=True)
catalog_extended.Year.value_counts()

Year
1977    17
1972     9
1975     8
1976     8
1971     6
1979     5
1970     4
1980     4
1981     3
1978     3
1973     3
1998     3
1991     2
1974     2
1983     2
2001     2
1989     2
1965     1
2003     1
2002     1
2000     1
1996     1
1999     1
1994     1
1988     1
1968     1
1995     1
1987     1
Name: count, dtype: int64

In [295]:
merged_cat = pd.merge(catalog_extended, uscities, on='City', how='left')
merged_cat.drop(['Document Body', 'city', 'state_name'], axis=1, inplace=True)
merged_cat['ID'] = merged_cat['ID'].astype('str')
merged_cat['url'] = "http://bondpapersproject.org/node/" + merged_cat['ID']
merged_cat

,ID,Title,City,Year,lat,lng,url
0,663,Speech about the upcoming presidential electio...,"New Orleans, Louisiana",1972,30.0687,-89.9288,http://bondpapersproject.org/node/663
1,662,(Draft) Speech about the upcoming presidential...,"New Orleans, Louisiana",1972,30.0687,-89.9288,http://bondpapersproject.org/node/662
2,661,(Draft) Speech about the upcoming presidential...,"New Orleans, Louisiana",1972,30.0687,-89.9288,http://bondpapersproject.org/node/661
3,618,Speech about the roles of black students in ad...,"Atlanta, Georgia",1972,33.7628,-84.4220,http://bondpapersproject.org/node/618
4,613,"Speech concerning political action, black peop...","Sanford, Florida",1972,28.7891,-81.2757,http://bondpapersproject.org/node/613
...,...,...,...,...,...,...,...
89,89,"Speech at the National Urban League Meeting, K...","Kansas City, Missouri",1974,39.1238,-94.5541,http://bondpapersproject.org/node/89
90,90,"Speech at the National Urban League Meeting, K...","Kansas City, Missouri",1974,39.1238,-94.5541,http://bondpapersproject.org/node/90
91,95,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",1973,33.7628,-84.4220,http://bondpapersproject.org/node/95
92,96,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",1973,33.7628,-84.4220,http://bondpapersproject.org/node/96


In [309]:
merged_cat['Year'] = merged_cat['Year'].astype('str')

In [311]:
mydict = {}
for year in merged_cat['Year'].unique():
    mydict[year] = merged_cat[merged_cat['Year'] == year]

filtered_dict = {k: v for k, v in mydict.items() if k in np.arange(1970,1980).astype('str')}
filtered_dict = dict(sorted(filtered_dict.items()))

print(filtered_dict)

{'1970':      ID                                              Title  \
6   610  Speech delivered to the ladies of Delta [Sigma...   
7   594  Working Outline for Julian Bond for a proposed...   
8   549  Speech delivered to the Congress of African Pe...   
15  470  Speech given at the Philadelphia Bar Associati...   

                         City  Year      lat      lng  \
6            Atlanta, Georgia  1970  33.7628 -84.4220   
7            Atlanta, Georgia  1970  33.7628 -84.4220   
8            Atlanta, Georgia  1970  33.7628 -84.4220   
15  Atlantic City, New Jersey  1970  39.3797 -74.4527   

                                      url  
6   http://bondpapersproject.org/node/610  
7   http://bondpapersproject.org/node/594  
8   http://bondpapersproject.org/node/549  
15  http://bondpapersproject.org/node/470  , '1971':      ID                                              Title  \
16  442  Commencement Address at Morgan State Universit...   
17  448  Speech concerning Atlanta, Georg

In [313]:
for k, v in filtered_dict.items():
    path = f'speeches_{str(k)}.csv'
    v.to_csv(path)

In [317]:
frames = filtered_dict.values()
speeches_70s = pd.concat(frames)
speeches_70s.to_csv('speeches_70s.csv')

Next week's puzzle (or maybe this weekend): try to get state abbreviations (CA, AL, etc.)

1. Create a second state string to extract based on. So, I would do an str.replace with a map to each state's abbreviation on the uscities dataset: would mean I had like "Huntsville, Alabama" but also "Huntsville, AL".
2. Then I can add both of those to the regex function and get it all in one column (some will be "Huntsville, AL" and others will be "Huntsville, Alabama")
4. From there, I can use the same replace map I made in the first place to loop through that column and convert all like "Huntsville, Alabama" to "Huntsville, AL".
5. Now, I can join based on that column in the uscities dataset, and go from there. Once I drop the columns, the entries will look exactly the same as before, just hopefully with even more information.
6. I think I am going to create a new Jupyter notebook for this and work on it on Sunday while I watch football. The new Jupyter notebook will allow me to clean up the mess I have made so far.

In [321]:
uscities

,city,state_name,lat,lng,City
0,New York,New York,40.6943,-73.9249,"New York, New York"
1,Los Angeles,California,34.1141,-118.4068,"Los Angeles, California"
2,Chicago,Illinois,41.8375,-87.6866,"Chicago, Illinois"
3,Miami,Florida,25.7840,-80.2101,"Miami, Florida"
4,Houston,Texas,29.7860,-95.3885,"Houston, Texas"
...,...,...,...,...,...
31115,Kohatk,Arizona,32.5781,-112.0032,"Kohatk, Arizona"
31116,Ironville,Pennsylvania,40.6586,-78.2155,"Ironville, Pennsylvania"
31117,Newkirk,New Mexico,35.0635,-104.2715,"Newkirk, New Mexico"
31118,Falcon Village,Texas,26.5652,-99.1341,"Falcon Village, Texas"


In [329]:
state_map = pd.read_csv('states.csv')
state_map

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [339]:
state_map_dict = dict(zip(state_map['State'], state_map['Abbreviation']))
uscities['state_abbrev'] = uscities['state_name'].map(state_map_dict)
uscities

,city,state_name,lat,lng,City,state_abbrev
0,New York,New York,40.6943,-73.9249,"New York, New York",NY
1,Los Angeles,California,34.1141,-118.4068,"Los Angeles, California",CA
2,Chicago,Illinois,41.8375,-87.6866,"Chicago, Illinois",IL
3,Miami,Florida,25.7840,-80.2101,"Miami, Florida",FL
4,Houston,Texas,29.7860,-95.3885,"Houston, Texas",TX
...,...,...,...,...,...,...
31115,Kohatk,Arizona,32.5781,-112.0032,"Kohatk, Arizona",AZ
31116,Ironville,Pennsylvania,40.6586,-78.2155,"Ironville, Pennsylvania",PA
31117,Newkirk,New Mexico,35.0635,-104.2715,"Newkirk, New Mexico",NM
31118,Falcon Village,Texas,26.5652,-99.1341,"Falcon Village, Texas",TX


In [359]:
uscities.rename({'City': 'city_full'}, axis = 1, inplace = True)

uscities['city_abbrev'] = uscities['city'] + ', ' + uscities['state_abbrev']
uscities

,city,state_name,lat,lng,city_full,state_abbrev,city_abbrev
0,New York,New York,40.6943,-73.9249,"New York, New York",NY,"New York, NY"
1,Los Angeles,California,34.1141,-118.4068,"Los Angeles, California",CA,"Los Angeles, CA"
2,Chicago,Illinois,41.8375,-87.6866,"Chicago, Illinois",IL,"Chicago, IL"
3,Miami,Florida,25.7840,-80.2101,"Miami, Florida",FL,"Miami, FL"
4,Houston,Texas,29.7860,-95.3885,"Houston, Texas",TX,"Houston, TX"
...,...,...,...,...,...,...,...
31115,Kohatk,Arizona,32.5781,-112.0032,"Kohatk, Arizona",AZ,"Kohatk, AZ"
31116,Ironville,Pennsylvania,40.6586,-78.2155,"Ironville, Pennsylvania",PA,"Ironville, PA"
31117,Newkirk,New Mexico,35.0635,-104.2715,"Newkirk, New Mexico",NM,"Newkirk, NM"
31118,Falcon Village,Texas,26.5652,-99.1341,"Falcon Village, Texas",TX,"Falcon Village, TX"


In [369]:

cities = uscities['city_full'].tolist() + uscities['city_abbrev'].tolist()

# Create regex pattern for cities
city_pattern = '|'.join([f'\\b{city}\\b' for city in cities])

In [377]:
# Extract the city and state from the Address column
catalog_extended = pd.read_csv('document-catalog_extended.csv')

catalog_extended['city_full'] = catalog_extended['Title'].str.extract(f'({city_pattern})', expand=False)

print(catalog_extended)

       ID                                              Title              Box  \
0    4375  Speech concerning Black Voters and Officials d...  Box 15 Folder 1   
1    3261  Enclosure: Julian Bond's revisions to text abo...  Box 14 Folder 6   
2     671       Speech File Material: Speech Fragment, 1971?  Box 12 Folder 3   
3     670  Undated Speech concerning Conditions of Black ...  Box 12 Folder 3   
4     668  Speech File Material: Undated Speech about the...  Box 12 Folder 3   
..    ...                                                ...              ...   
433    88  Speech on the health of the United States educ...   Box 4 Folder 8   
434     4            "Collision Course in a Divided America"              NaN   
435     7  Address at the "New Directions in Civil Rights...              NaN   
436    11  DUPLICATE DO NOT PUBLISH: "Civil Rights and Wr...              NaN   
437    12  Speech concerning the black college: its effec...              NaN   

     Document Length Workfl

In [379]:
catalog_extended.drop(['Box', 'Document Length',
                  'Image Filename', 'Image Identifier', 'Image URL'], axis=1, inplace=True)

In [387]:
filtered_df = catalog_extended[catalog_extended['city_full'].notnull()]
filtered_df

,ID,Title,Workflow Stage,Document Body,city_full
4,668,Speech File Material: Undated Speech about the...,Cataloged,NaN,"Chicago, Illinois"
8,663,Speech about the upcoming presidential electio...,Early Access,The election approaching on November seventh i...,"New Orleans, Louisiana"
9,662,(Draft) Speech about the upcoming presidential...,Early Access,Page 1\nJULIAN BOND\nThe election approaching ...,"New Orleans, Louisiana"
10,661,(Draft) Speech about the upcoming presidential...,Early Access,[Handwritten on legal pad paper.]\n[The follow...,"New Orleans, Louisiana"
11,660,Speech concerning the trial of the RNA–11 (Rep...,Early Access,"Writing of another political trial, his own, D...","Jackson, MS"
...,...,...,...,...,...
415,90,"Speech at the National Urban League Meeting, K...",Early Access,The quadrennial occasion for American reaction...,"Kansas City, Missouri"
420,95,Testimony before the Commission on Delegate Se...,Early Access,I am pleased to testify on behalf of the refor...,"Atlanta, Georgia"
421,96,Testimony before the Commission on Delegate Se...,Early Access,"""Quotas"" is the code word for 1972. Like most ...","Atlanta, Georgia"
422,97,Testimony before the Commission on Delegate Se...,Early Access,"""Quotas"" is the code word for 1972. Like most ...","Atlanta, Georgia"


In [389]:
for abbrev, full_name in state_map_dict.items():
    filtered_df['city_full'] = filtered_df['city_full'].str.replace(full_name, abbrev)

print(filtered_df)

      ID                                              Title Workflow Stage  \
4    668  Speech File Material: Undated Speech about the...      Cataloged   
8    663  Speech about the upcoming presidential electio...   Early Access   
9    662  (Draft) Speech about the upcoming presidential...   Early Access   
10   661  (Draft) Speech about the upcoming presidential...   Early Access   
11   660  Speech concerning the trial of the RNA–11 (Rep...   Early Access   
..   ...                                                ...            ...   
415   90  Speech at the National Urban League Meeting, K...   Early Access   
420   95  Testimony before the Commission on Delegate Se...   Early Access   
421   96  Testimony before the Commission on Delegate Se...   Early Access   
422   97  Testimony before the Commission on Delegate Se...   Early Access   
426   80  Speech made before the Hungry Club [Forum, Atl...   Early Access   

                                         Document Body         

C:\Users\charl\AppData\Local\Temp\ipykernel_23116\1392622918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['city_full'] = filtered_df['city_full'].str.replace(full_name, abbrev)


hmmm. Setting With Copy warning is decidedly a concern, but not sure how big of a concern it ought to be. The dataset's so small, and I can always just filter the way I had filtered before... trying to remember why I want to keep it. Every doc has a workflow stage, right?

Oh also none of this will matter once the other interns start doing location tagging. Duh. Main deliverable should just be a way to tag places. Sigh.

In [399]:
filtered_df.loc[415,:]

ID                                                               90
Title             Speech at the National Urban League Meeting, K...
Workflow Stage                                         Early Access
Document Body     The quadrennial occasion for American reaction...
city_full                                     Kansas City, Missouri
Name: 415, dtype: object